## Scraping $ada Tweets using snscrape

In this notebook we will be scraping Tweets from Twitter's API to extract all tweets mentioning '\\$ada' in the period between 01-01-2019 (00:00 UTC) to 28-02-2021 (23:59 UTC). While scraping Twitter for the word Cardano would have possibly returned too much noice, using '\\$ada' is a saver option. '\\$ada' can be seen as '#ada', while the '\\$' icon is a hashtag for financial markets. Twitter's API is not sensitive to lower or upper case variations of the word.

This notebook and the code below are based on the developers version of the snscrape package (https://github.com/JustAnotherArchivist/snscrape), check their github for more information about the developers version of snscrape.

The process of scraping Tweets will be done by iteratively requesting individual samples until a threshold date is reached. Each request returns new Tweets (including additional data) from a specified point in time. A new request will take the date from the most recent Tweet from the last request and request new tweets from that date on. This causes partial overlapping if the end tweet of the previous call is not the last one of that day, while time can not be specified within the request. To overcome this, Panda's drop_duplicates method is used. All requested are: stored as json file, opened and converted to df and saved to a csv file per all Tweets corresponding to a specific month.

In [1]:
# Run the pip install command below if you don't already have the library. You might need to restart your program for the package to work.
#!pip3 install git+https://github.com/JustAnotherArchivist/snscrape.git

import os
import pandas as pd

In [7]:
# The following list contains the names of the csv files corresponding to the months of the search period.
# These will later be used to save the data corresponding to that month.
csv_names_list = ['T_ada_march_18.csv', 'T_ada_april_18.csv', 'T_ada_may_18.csv', 'T_ada_june_18.csv', 'T_ada_july_18.csv', 'T_ada_august_18.csv',\
                  'T_ada_september_18.csv', 'T_ada_october_18.csv', 'T_ada_november_18.csv', 'T_ada_december_18.csv', 'T_ada_january_19.csv',\
                  'T_ada_february_19.csv', 'T_ada_march_19.csv', 'T_ada_april_19.csv', 'T_ada_may_19.csv', 'T_ada_june_19.csv', 'T_ada_july_19.csv',\
                  'T_ada_august_19.csv', 'T_ada_september_19.csv', 'T_ada_october_19.csv', 'T_ada_november_19.csv', 'T_ada_december_19.csv',\
                  'T_ada_january_20.csv', 'T_ada_febrauri_20.csv', 'T_ada_march_20.csv', 'T_ada_april_20.csv', 'T_ada_may_20.csv', 'T_ada_june_20.csv',\
                  'T_ada_july_20.csv', 'T_ada_august_20.csv', 'T_ada_september_20.csv', 'T_ada_october_20.csv', 'T_ada_november_20.csv',\
                  'T_ada_december_20.csv', 'T_ada_january_21.csv', 'T_ada_february_21.csv', 'T_ada_march_21.csv', 'T_ada_april_21.csv', 'T_ada_may_21.csv']

# The following lists contain the start and end dates
start_dates = ["2018-03-01", "2018-04-01", "2018-05-01", "2018-06-01", "2018-07-01", "2018-08-01", "2018-09-01",\
               "2018-10-01", "2018-11-01", "2018-12-01",\
               "2019-01-01", "2019-02-01", "2019-03-01", "2019-04-01", "2019-05-01", "2019-06-01", "2019-07-01", "2019-08-01", "2019-09-01",\
               "2019-10-01", "2019-11-01", "2019-12-01",\
               "2020-01-01", "2020-02-01", "2020-03-01", "2020-04-01", "2020-05-01", "2020-06-01", "2020-07-01", "2020-08-01", "2020-09-01",\
               "2020-10-01", "2020-11-01", "2020-12-01",\
               "2021-01-01", "2021-02-01", "2021-03-01", "2021-04-01", "2021-05-01"]
end_dates = ["2018-04-01", "2018-05-01", "2018-06-01", "2018-07-01", "2018-08-01", "2018-09-01",\
             "2018-10-01", "2018-11-01", "2018-12-01",\
             "2019-01-01", "2019-02-01", "2019-03-01", "2019-04-01", "2019-05-01", "2019-06-01", "2019-07-01", "2019-08-01", "2019-09-01",\
             "2019-10-01", "2019-11-01", "2019-12-01",\
             "2020-01-01", "2020-02-01", "2020-03-01", "2020-04-01", "2020-05-01", "2020-06-01", "2020-07-01", "2020-08-01", "2020-09-01",\
             "2020-10-01", "2020-11-01", "2020-12-01",\
             "2021-01-01", "2021-02-01", "2021-03-01", "2021-04-01", "2021-05-01", "2021-06-01"] 

# The one day Timedelta object will later be used to move the end date.
ONE_DAY = pd.Timedelta(1, unit = 'd')

In [8]:
len(start_dates) == len(csv_names_list)

True

In [9]:
# The for-loop goes iteratively through the start_dates and end_dates lists.
for x in range(len(start_dates)):
    since_date = start_dates[x]
    until_date = end_dates[x]
    end_date = str(pd.Timestamp.date(pd.Timestamp(since_date) + ONE_DAY))
    count = 0

    tweet_count = 15000
    text_query = "$ada"
# Using OS library to call CLI commands in Python
    os.system('snscrape --jsonl --max-results {} --since {} twitter-search "{} until:{}"> text-query-tweets.json'.format(tweet_count, since_date, text_query, until_date))

    tweets_dot = pd.read_json('text-query-tweets.json', lines=True)
    new_date = tweets_dot.iloc[-1,1]
    new_date = str(pd.Timestamp.date(new_date))
# While one call of 15.000 Tweets is most often not enough to cover all Tweets within a month, the following while-loop repeats the previous request.
# This is done until the end_date is reached.
    while count == 0:
        if end_date > new_date:
            count += 1
    
        until_date = new_date

        os.system('snscrape --jsonl --max-results {} --since {} twitter-search "{} until:{}"> text-query-tweets.json'.format(tweet_count, since_date, text_query, until_date))

        tweets_dot = tweets_dot.append(pd.read_json('text-query-tweets.json', lines=True))
    
        new_date = tweets_dot.iloc[-1,1]
        new_date = str(pd.Timestamp.date(new_date))
# When the While-loop is completed, all the Tweets from a specific month are cleaned from duplicates and saved to the corresponding csv file.
    tweets_dot = tweets_dot.drop_duplicates(subset = 'id')
    tweets_dot.to_csv(csv_names_list[x], sep=',', index=False)
    print(f"Data loaded and saved until {end_date}")


Data loaded and saved until 2018-03-02
Data loaded and saved until 2018-04-02
Data loaded and saved until 2018-05-02
Data loaded and saved until 2018-06-02
Data loaded and saved until 2018-07-02
Data loaded and saved until 2018-08-02
Data loaded and saved until 2018-09-02
Data loaded and saved until 2018-10-02
Data loaded and saved until 2018-11-02
Data loaded and saved until 2018-12-02
Data loaded and saved until 2019-01-02
Data loaded and saved until 2019-02-02
Data loaded and saved until 2019-03-02
Data loaded and saved until 2019-04-02
Data loaded and saved until 2019-05-02
Data loaded and saved until 2019-06-02
Data loaded and saved until 2019-07-02
Data loaded and saved until 2019-08-02
Data loaded and saved until 2019-09-02
Data loaded and saved until 2019-10-02
Data loaded and saved until 2019-11-02
Data loaded and saved until 2019-12-02
Data loaded and saved until 2020-01-02
Data loaded and saved until 2020-02-02
Data loaded and saved until 2020-03-02
Data loaded and saved unt